


Cross Validation & Naive Bayes Lab - SMS Spam Classification
===============
* orignally developed by Ankit Jain
* modified by Justin Breucop
* modified by Dylan Hercher

Data source: https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection

# Section 1)

## Cross Validation from Scratch

Let's build it the function together! The steps to cross validation are:
1. Randomly separate your training set into _k_ groups
2. For each group _k_:
>1. Train your model on the other groups
>2. Score your model using group _k_ as validation
>3. Save your score and move to your next group

3. Add your _k_ scores and divide by _k_ to get your average score

In [1]:
# Importing Packages 
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier

ValueError: unknown locale: UTF-8

In [ ]:
model = KNeighborsClassifier(10)

df = pd.DataFrame.from_csv('../data/titanic-train.csv',index_col=None)[['Age','Pclass','SibSp','Survived']].dropna()
to_predict = "Survived"
features=['Age','Pclass','SibSp']
data = df[features]
label = df[to_predict]
folds=5

Let's build in class!

In [ ]:
#Function here:

def cross_validate_df(data,label,model,k):
    """Return Average score across k iterations
    Parameters
    -----------
    data : DataFrame with features to use in X
    label : Series with target y
    model : ML Model to use
    k : int number of iterations
    """
    train_slices = []
    test_slices = []
    positions = data.index.values

    # Remember this technique? We're going to 
    # shuffle our data in place using NumPy
    np.random.shuffle(positions)

    for i in range(k):

        # When creating and evaluating a loop, it's important to understand
        # the loop at 3 points: 
        # initial iteration (i = 0)
        # steady state (0 < i < end) assuming it's running smoothly
        # exit (i = end)

        section_start = i*len(data)/k
        section_end = (i+1)*len(data)/k
        test_slices.append(positions[ section_start:section_end ])

        # We now want the rest of the data, slicing up to the start
        # and combining that with a slice from the test slice end to
        # the end of the dataframe

        train_p1 = positions[ :section_start ]
        train_p2 = positions[ section_end: ]
        train_combo = np.concatenate([train_p1,train_p2])
        train_slices.append(train_combo)

    scores = 0

    for train_sl, test_sl in zip(train_slices,test_slices):

        model.fit(data.loc[train_sl],label.loc[train_sl])
        k_score = model.score(data.loc[test_sl],label.loc[test_sl])
        scores += k_score
        print k_score
    return scores/k

In [ ]:
# Once Complete test your function here
cross_validate_df(data,label,model,5)

### Built-in Cross Validation
There is also a very simple cross validation function provided by sklearn

In [ ]:
from sklearn.cross_validation import cross_val_score

# Results of using the built-in cross validation
# Note: Default is Kfold, but within sklearn.cross_validation
# there are many types of validation that can be used
cross_val_score(model, data, label, cv=3)

In [ ]:
cross_val_score(model, data, label, cv=3)

In [ ]:
np.mean(cross_val_score(model, data, label, cv=3))

# Section 2)
## Naive Bayes and SMS Spam Classification

In [ ]:
import numpy as np
import pandas as pd

from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

In [ ]:
## READING IN THE DATA
df = pd.DataFrame.from_csv("../data/SMSSpamCollection.tsv",sep='\t',header=0,index_col=None)

In [ ]:
# examine the data
df.head(3)

In [ ]:
df[df.label=='spam'].head(10)

In [ ]:
df.label.value_counts()

In [ ]:
df.msg.describe()

In [ ]:
# Convert the label into a binary variable
# Remember the map function we learned before?
df['label'] = df.label.map({'ham': 0 , 'spam':1})

In [ ]:
df.head()

In [ ]:
# split into training and testing sets by calling sklearn lib
# by default, the data set is split into 0.75 (training) and 0.25 (testing)
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.msg, df.label, random_state=1)

In [ ]:
print X_train.shape
print X_train

In [ ]:
X_test.shape

Now we need to convert the text into feature vectors which can be used for machine learning purposes.
We will use the scikit function of CountVectorizer to 'convert text into a matrix of token counts'

 http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

#### Lets try a simple example

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# start with a simple example
train_simple = ['call you tonight',
                'Call me a cab',
                'please call me... PLEASE!']

In [ ]:
# learn the 'vocabulary' of the training data
vect = CountVectorizer(decode_error = 'ignore')
vect.fit(train_simple)
vect.get_feature_names()

In [ ]:
# transform training data into a 'document-term matrix'
train_simple_dtm = vect.transform(train_simple)
train_simple_dtm.toarray()

In [ ]:
# We can see how we've adjusted our data easily!
# examine the vocabulary and document-term matrix together
print train_simple
    
pd.DataFrame(train_simple_dtm.toarray(), columns=vect.get_feature_names())

In [ ]:
# transform testing data into a document-term matrix (using existing vocabulary)
test_simple = ["please don't call me"]
test_simple_dtm = vect.transform(test_simple)

test_simple_dtm.toarray()
pd.DataFrame(test_simple_dtm.toarray(), columns=vect.get_feature_names())

#### Question:  How does the above test_simple show how things can go wrong?

#### Exercise: Using the dataset below
   * Vectorize the text
   * Store the results in a DataFrame
   * Show word counts (hint: one dataframe describer can do this)
   * Transform the test text

In [ ]:
train_exp = ['where is my taco?',
                'did I eat the taco',
                'I can easily eat my way through that whole box of tacos!',
                'I think way too much about tacos, huh',
                'taco, taco, taco!!!'                
               ]
test_exp = [
    'where did he go?', 'how long did the whole thing last', 'lets go eat one taco or multiple tacos'
]

Vectorize the text

In [ ]:
vect = CountVectorizer(decode_error = 'ignore')
vect.fit(train_exp)
train_exp_dtm = vect.transform(train_simple)

Store the results in a DataFrame

In [ ]:
train_exp_df = pd.DataFrame(train_exp_dtm.toarray(), columns=vect.get_feature_names())
train_exp_df.head(2)

Show word counts (hint: one dataframe describer can do this)

In [ ]:
# Each word is a column
# See how describe really just returns a dataframe, Ican use head
train_exp_df.describe().head(1)

Transform the test text

In [ ]:
test_exp_dtm = vect.transform(test_exp)

test_exp_dtm.toarray()

## Vectorizing our SMS Dataset

Returning to our SMS Spam Dataset

In [ ]:
X_train

In [ ]:
# instantiate the vectorizer ( use variable name as vect)
vect = CountVectorizer(decode_error = 'ignore')
vect.fit(X_train)
vect.get_feature_names()

In [ ]:
# transform testing data into a document-term matrix: Use Variable name as test_dtm
train_dtm = vect.transform(X_train)
test_dtm = vect.transform(X_test)
print test_dtm

In [ ]:
# Get the length  and names of the feature names
train_features = vect.get_feature_names()
len(train_features)

In [ ]:
train_features[:50]

In [ ]:
train_features[-50:]

In [ ]:
# convert train_dtm to a regular array
train_arr = train_dtm.toarray()
train_arr

In [ ]:

# Revisit Numpy
arr = np.array([[1, 2, 3, 4], [5, 6, 7, 8]])
print arr[0, 0]
print arr[1, 3]
print arr[0, :]
print arr[:, 0]
print np.sum(arr)
print np.sum(arr,axis = 0)
print np.sum(arr,axis = 1)




In [ ]:
# exercise: calculate the number of tokens in the 0th message in train_arr
print np.sum(train_arr[0,:])

In [ ]:

# exercise: count how many times the 0th token appears across ALL messages in train_arr
print np.sum(train_arr[:,0])

In [ ]:
# exercise: count how many times EACH token appears across ALL messages in train_arr
print np.sum(train_arr, axis=0)

In [ ]:
# exercise: create a DataFrame of tokens with their counts.
train_token_counts = pd.DataFrame({'token':train_features, 'count':np.sum(train_arr, axis=0)})
train_token_counts.sort_index(by='count', ascending=False)

Let's build the model with Naive Bayes Now

http://scikit-learn.org/stable/modules/naive_bayes.html

In [ ]:
# train a Naive Bayes model using train_dtm
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(train_dtm, y_train)

In [ ]:
# make predictions on test data using test_dtm
preds = nb.predict(test_dtm)
preds

In [ ]:
# compare predictions to true labels
from sklearn import metrics

print metrics.accuracy_score(y_test, preds)
print metrics.confusion_matrix(y_test, preds)
# confusion matrix: http://en.wikipedia.org/wiki/Confusion_matrix

In [ ]:
# exercise: show the message text for the false positives
X_test[(y_test == 0) & (preds == 1)]

In [ ]:
# exercise: show the message text for the false negatives
X_test[y_test > preds]
# or
X_test[(y_test == 1) & (preds == 0)]

In [ ]:
## USING ALL DATA AND CROSS-VALIDATION, run NB again
df = pd.DataFrame.from_csv("../data/SMSSpamCollection.tsv",sep='\t',header=0,index_col=None)
df.label = df.label.map({'ham': 0 , 'spam':1})

X_train, X_test, y_train, y_test = train_test_split(df.msg, df.label, random_state=1)

vect = CountVectorizer(decode_error = 'ignore')

vect.fit(X_train)
vect.get_feature_names()

train_dtm = vect.transform(X_train)
test_dtm = vect.transform(X_test)

In [ ]:
df.head(1)

In [ ]:
from sklearn.cross_validation import cross_val_score
nb = MultinomialNB()

vect = CountVectorizer(decode_error='ignore')

vect.fit(df.msg)

X_dtm = vect.transform(df.msg)
y = df.label

cross_val_score(nb, X_dtm, y, cv=5)

In [ ]:
## EXERCISE: CALCULATE THE 'SPAMMINESS' OF EACH TOKEN

# create separate DataFrames for ham and spam ( df_ham and df_spam)
df_ham = df[df.label==0]
df_spam = df[df.label==1]

In [ ]:
# learn the vocabulary of ALL messages and save it
vect.fit(df.msg)
all_features = vect.get_feature_names()

In [ ]:
# create document-term matrix of ham, then convert to a regular array
ham_dtm = vect.transform(df_ham.msg)
ham_arr = ham_dtm.toarray()

In [ ]:
# create document-term matrix of spam, then convert to a regular array
spam_dtm = vect.transform(df_spam.msg)
spam_arr = spam_dtm.toarray()

In [ ]:
# count how many times EACH token appears across ALL messages in ham_arr
ham_counts = np.sum(ham_arr, axis=0)
ham_counts

In [ ]:
# count how many times EACH token appears across ALL messages in spam_arr
spam_counts = np.sum(spam_arr, axis=0)
spam_counts

In [ ]:
# create a DataFrame of tokens with their separate ham and spam counts
all_token_counts = pd.DataFrame({'token':all_features, 'ham':ham_counts, 'spam':spam_counts})
all_token_counts.head()

In [ ]:
# add one to ham counts and spam counts so that ratio calculations (below) make more sensse
all_token_counts['ham'] = all_token_counts.ham + 1
all_token_counts['spam'] = all_token_counts.spam + 1

In [ ]:
# calculate ratio of spam-to-ham for each token
all_token_counts['spam_ratio'] = all_token_counts.spam / all_token_counts.ham
all_token_counts.sort_index(by='spam_ratio',ascending = False )

In [ ]:
# advanced: implement your own naive bayes classifier
